## Setup

In [1]:
# Install packages and frameworks

import tensorflow as tf
import os

# expecting 2.11
# if 2.7, than logging errors will show "Cleanup called..."
print(tf.__version__)

2023-05-25 04:30:35.822774: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 04:30:35.939462: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-25 04:30:36.871377: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-05-25 04:30:36.871448: W tensorflow/

2.11.1


In [2]:
# scratch directory is apart of the .gitignore to ensure it is not committed to git
%env SCRATCH=../scratch
! [ -e "${SCRATCH}" ] || mkdir -p "${SCRATCH}"

scratch_path = os.environ.get('SCRATCH', './scratch')

env: SCRATCH=../scratch


# Load the saved datasets

The TFRecord format is a simple format for storing a sequence of binary records.

In [3]:
path = scratch_path + '/tf_datasets/train/'
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF

train_ds = tf.data.Dataset.load(path)
train_ds = train_ds.with_options(options)

train_ds.element_spec

2023-05-25 04:30:38.716712: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-25 04:30:38.743082: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-25 04:30:38.744872: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-25 04:30:38.747041: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operati

(TensorSpec(shape=(None, 96, 96, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))

In [4]:
path = scratch_path + '/tf_datasets/validate/'
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF

validation_ds = tf.data.Dataset.load(path)
validation_ds = validation_ds.with_options(options)

validation_ds.element_spec

(TensorSpec(shape=(None, 96, 96, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))

In [5]:
path = scratch_path + '/tf_datasets/test/'
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF

test_ds = tf.data.Dataset.load(path)
test_ds = test_ds.with_options(options)

test_ds.element_spec

(TensorSpec(shape=(None, 96, 96, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))

## Configure the datasets for performance

Let's make sure to use buffered prefetching so we can yield data from disk without having I/O become blocking. These are two important methods you should use when loading data.

1. `Caching` a dataset, either in memory or on local storage. This will save some operations (like file opening and data reading) from being executed during each epoch.
1. `Prefetching` overlaps the preprocessing and model execution of a training step. While the model is executing training step s, the input pipeline is reading the data for step s+1. Doing so reduces the step time to the maximum (as opposed to the sum) of the training and the time it takes to extract the data.

[Optimising your input pipeline performance with tf.data (part 1)](https://towardsdatascience.com/optimising-your-input-pipeline-performance-with-tf-data-part-1-32e52a30cac4)

# Check the device spec

In [6]:
# display physical devices
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Set the training strategy based on your devices

## One Device

Typical usage of this strategy could be testing your code with the tf.distribute.Strategy API before switching to other strategies which actually distribute to multiple devices/machines.

In [7]:
#  example, single device
#device="/cpu:0"
device="/GPU:0"

strategy = tf.distribute.OneDeviceStrategy(device=device)

# Create the model

## Set some variables

In [8]:
# set variables for consistency
img_height = 96              # desired height
img_width = 96               # desired width
batch_size = 32              # batch inputs in 32
seed_train_validation = 42   # Must be same for train_ds and val_ds
validation_split = 0.3       # move 30% of the data into validation
class_names = ['left', 'right']

dataFormat="channels_last"
num_classes = len(class_names)
inputShape=(img_height, img_width, 1)
chanDim = -1

tf.keras models are optimized to make predictions on a batch, or collection, of examples at once. Accordingly, even though you're using a single image, you need to add it to a list:

## Setup the layers

The Sequential model consists of three convolution blocks (tf.keras.layers.Conv2D) with a max pooling layer (tf.keras.layers.MaxPooling2D) in each of them. There's a fully-connected layer (tf.keras.layers.Dense) with 128 units on top of it that is activated by a ReLU activation function ('relu'). This model has not been tuned in any way—the goal is to show you the mechanics using the datasets you just created. To learn more about image classification, visit the Image classification tutorial.

Here's an example of Python code using Keras to create a sequential classification model that accepts 96x96x1 input images.

In this example, we start with three convolutional layers, each followed by a max pooling layer to downsample the input. Then, we flatten the output from the convolutional layers and add two fully connected layers. Finally, we add an output layer with the number of classes and compile the model using an optimizer (e.g., Adam) and a loss function (e.g., categorical cross-entropy).

Make sure to replace num_classes with the actual number of classes in your classification problem.
```
import tensorflow as tf    

model = tf.keras.Model(...)

# Run training on GPU
with tf.device('/gpu:0'):
    model.fit(...)

# Run inference on CPU
with tf.device('/cpu:0'):
    model.predict(...)
    
```

# Set up the layers
The basic building block of a neural network is the layer. Layers extract representations from the data fed into them. Hopefully, these representations are meaningful for the problem at hand.

Most of deep learning consists of chaining together simple layers. Most layers, such as tf.keras.layers.Dense, have parameters that are learned during training.

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow import keras
from keras.utils import plot_model

with strategy.scope():
    fp_3_layer_model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=inputShape),
        tf.keras.layers.Reshape(target_shape=inputShape),

        # layer 1
        tf.keras.layers.Conv2D(96, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        # layer 2
        tf.keras.layers.Conv2D(96, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        # layer 3
        tf.keras.layers.Conv2D(96, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(num_classes),
        tf.keras.layers.Softmax()
    ])

In [14]:
# Visualize the model
plot_model(fp_3_layer_model, to_file='fp_3_layer_model.png', show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


# Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:

Loss function —This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.
Optimizer —This is how the model is updated based on the data it sees and its loss function.
Metrics —Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.

In [15]:
fp_3_layer_model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(
                  from_logits=False,
                  label_smoothing=0.0,
                  axis=-1,
                  #reduction=losses_utils.ReductionV2.AUTO,
                  name='categorical_crossentropy'),
              metrics=['accuracy'])

fp_3_layer_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 96, 96, 1)         0         
                                                                 
 conv2d_5 (Conv2D)           (None, 94, 94, 96)        960       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 47, 47, 96)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 45, 45, 96)        83040     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 22, 22, 96)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 20, 20, 96)       

# Train the model
Training the neural network model requires the following steps:

Feed the training data to the model. In this example, the training data is in the train_images and train_labels arrays.
The model learns to associate images and labels.
You ask the model to make predictions about a test set—in this example, the test_images array.
Verify that the predictions match the labels from the test_labels array.

In [17]:
with tf.device(device):
    fp_3_layer_model.fit(train_ds, epochs=10, validation_data=validation_ds)

Epoch 1/10


2023-05-25 04:32:53.582160: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 103.55MiB (rounded to 108576768)requested by op sequential_1/conv2d_5/Relu
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-25 04:32:53.582203: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-05-25 04:32:53.582217: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 45, Chunks in use: 45. 11.2KiB allocated for chunks. 11.2KiB in use in bin. 1.5KiB client-requested in use in bin.
2023-05-25 04:32:53.582228: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 24, Chunks in use: 24. 12.0KiB allocated for chunks. 12.0KiB in use in bin. 9.0KiB client-requested in use in bin.
2023-05-25 04:32:53.582236: I tenso

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential_1/conv2d_5/Relu' defined at (most recent call last):
    File "/usr/lib64/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib64/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/opt/app-root/lib64/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/app-root/lib64/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/app-root/lib64/python3.9/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/opt/app-root/lib64/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib64/python3.9/asyncio/base_events.py", line 601, in run_forever
      self._run_once()
    File "/usr/

# Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [ ]:
test_loss, test_acc = fp_3_layer_model.evaluate(test_ds, verbose=2)

# Print Accuracy

In [ ]:
print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)